[inspiration](https://twitter.com/__mharrison__/status/1638918620532322306/photo/1)

<img src="styling.jpeg" width="600"/>

### Sleuth the OG data source

A quick Bing (!) search for 'matt harrison dirty devil data' reveals a text file in [his repo](https://github.com/mattharrison/datasets/tree/master/data).

A quick way to download just the file is to issue the following in a terminal:

```curl -o styling_dirtydevil.txt https://raw.githubusercontent.com/mattharrison/datasets/master/data/dirtydevil.txt```

### Load raw data into `pandas` dataframe

In [1]:
import pandas as pd

In [18]:
dd = (pd.read_csv('./styling_dirtydevil.txt', delimiter='\t', skiprows=34, parse_dates=['datetime'], low_memory=False)
    .drop(columns=['agency_cd', 'site_no'])
    .rename(columns={
        'tz_cd': 'tz',
        '144166_00060': 'discharge',
        '144166_00060_cd': 'discharge_quals',
        '144167_00065': 'gage_ht',
        '144167_00065_cd': 'gage_ht_quals'
    })
    .set_index('datetime')
)

In [26]:
agg_flow = (dd
    [['discharge', 'gage_ht']]
    .groupby(pd.Grouper(freq='MS'))
    .mean()
)

In [60]:
(agg_flow 
    .iloc[:12]
    .reset_index()
    .assign(discharge2 = lambda df_: df_.discharge,
            discharge3 = lambda df_: df_.discharge,
            discharge4 = lambda df_: df_.discharge)
    .style
    # everything that follows will apply to  a Styler object, NOT the dataframe
    .format({'datetime': '{:%Y/%m/%d}',
             'discharge': '{:.1f}'},
             na_rep='Missing')
    .bar(subset='discharge', color='#c07fef', vmax=agg_flow.discharge.quantile(.95))
    .highlight_null()
    .highlight_max(axis=0, color='green')
    .background_gradient(axis=0, cmap='Blues', subset='discharge3', vmin=0, vmax=125)
    .set_caption('Dude Displays Dirty Devil Data')
    .set_properties(**{'background-color': '#999'}, subset='datetime')
    .applymap(lambda val: f'color: "grey"; opacity: 80%; background-color:{"#4589ae" if val > 0 else "#c07fef"}',
              subset=['discharge', 'discharge4'])
    .hide(axis='index')
    # .set_table_styles breaks stick headers if it is after (???)
    # .set_table_styles([{'selector': 'td_hover', 'props': 'background-color: pink; font-size:14pt;'}])
    # .set_sticky(axis='columns')
)

datetime,discharge,gage_ht,discharge2,discharge3,discharge4
2001/05/01,48.9,Missing,48.870200,48.870200,48.870200
2001/06/01,22.9,Missing,22.946759,22.946759,22.946759
2001/07/01,18.0,Missing,18.012500,18.012500,18.012500
2001/08/01,248.0,Missing,248.004398,248.004398,248.004398
2001/09/01,29.2,Missing,29.152574,29.152574,29.152574
2001/10/01,52.1,Missing,52.073958,52.073958,52.073958
2001/11/01,102.2,Missing,102.209239,102.209239,102.209239
2001/12/01,117.7,Missing,117.723214,117.723214,117.723214
2002/01/01,134.6,Missing,134.630515,134.630515,134.630515
2002/02/01,130.6,Missing,130.555804,130.555804,130.555804
